In [1]:
import os
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def cartesian_product_transpose(*arrays):
    broadcastable = np.ix_(*arrays)
    broadcasted = np.broadcast_arrays(*broadcastable)
    rows, cols = np.prod(broadcasted[0].shape), len(broadcasted)
    dtype = np.result_type(*arrays)

    out = np.empty(rows * cols, dtype=dtype)
    start, end = 0, rows
    for a in broadcasted:
        out[start:end] = a.reshape(-1)
        start, end = end, end + rows
    return out.reshape(cols, rows).T

def generate_weight(min_weight=0.1, interval=0.1, weight_num=5, column_names=None):
    arrays = [np.arange(min_weight, 1+interval, interval)] * weight_num
    all_weight = cartesian_product_transpose(*arrays)
    all_weight_c_sum = all_weight.sum(1)
    sel_weight = all_weight[all_weight_c_sum==1]
    if column_names is None:
        column_names = [f'w{i+1}' for i in range(weight_num)]
    data = pd.DataFrame(sel_weight, columns=column_names, dtype=np.float32)
    index = data.mul(10).astype(int).astype(str).apply(lambda x: '_'.join(x), axis=1)
    data.index=index
    return data

In [3]:
data = pd.read_csv('../data/NWP.csv', parse_dates=['gtime', 'time'], index_col=['sta', 'gtime', 'ltime'])[['pm25', 'rh2', 't2', 'u10', 'v10']]
data.head()

pm25   rh2     t2  u10  v10
sta   gtime               ltime                              
1008A 2016-01-01 20:00:00 0      172.9  84.4  269.2 -0.3 -0.2
                          1      183.4  82.0  269.8 -0.2 -0.8
                          2      191.1  78.7  270.3 -0.2 -1.1
                          3      121.7  68.6  271.2 -0.1 -0.8
                          4       47.1  61.0  272.1 -1.2 -2.1

In [4]:
test_date = '2019-01-01 20:00'
train = data.query('gtime<@test_date')
test = data.query('gtime==@test_date')
train.head()

pm25   rh2     t2  u10  v10
sta   gtime               ltime                              
1008A 2016-01-01 20:00:00 0      172.9  84.4  269.2 -0.3 -0.2
                          1      183.4  82.0  269.8 -0.2 -0.8
                          2      191.1  78.7  270.3 -0.2 -1.1
                          3      121.7  68.6  271.2 -0.1 -0.8
                          4       47.1  61.0  272.1 -1.2 -2.1

# 变量权重

In [7]:
train.index.levels.sta

AttributeError: 'FrozenList' object has no attribute 'sta'

In [85]:
def calc_weight_step_one(train:pd.DataFrame, test:pd.DataFrame, vweight=None):
#      assume coords order : sta gtime vweight ltime
    nsta = train.index.unique('sta').size
    ngtime = train.index.unique('gtime').size
    nltime = train.index.unique('ltime').size
    
    train_index = train.index
    train_columns=train.columns
    ncolumns=train_columns.size
    
    if vweight is None:
        vweight = pd.DataFrame(np.ones((1,ncolumns)), 
                               columns=train_columns,
                               index=['_'.join(['1'] * ncolumns)])
        vweight.index.name='vweight'
    nweight = vweight.index.unique('vweight').size
    
    train_np = train.to_numpy().reshape(nsta, ngtime, nltime, ncolumns)
    test_np = test.to_numpy().reshape(nsta, 1, nltime, ncolumns)
    distance = pd.DataFrame(
        (train_np - test_np).reshape(-1, ncolumns), 
        index=train_index,
        columns=train_columns
    )
    return distance

def calc_train_test_delta(train:pd.DataFrame, test:pd.DataFrame):
    nsta = train.index.unique('sta').size
    ngtime = train.index.unique('gtime').size
    nltime = train.index.unique('ltime').size
    train_index = train.index
    train_columns=train.columns
    ncolumns=train_columns.size
    train_np = train.to_numpy().reshape(nsta, ngtime, nltime, ncolumns)
    test_np = test.to_numpy().reshape(nsta, 1, nltime, ncolumns)
    distance = pd.DataFrame(
        (train_np - test_np).reshape(-1, ncolumns), 
        index=train_index,
        columns=train_columns
    )
    return distance.to_xarray()
    


def calc_weight_delta(train:pd.DataFrame, test:pd.DataFrame, vweight=None):
    train_test_delta = calc_train_test_delta(train, test)
    
    if vweight is None:
        vweight = pd.DataFrame(np.ones((1,ncolumns)), 
                               columns=train_columns,
                               index=['_'.join(['1'] * ncolumns)])
        vweight.index.name='vweight'
    
    if isinstance(vweight, pd.DataFrame):
        vweight = vweight.to_xarray()

    distance_weight = train_test_delta * vweight
    
    return distance_weight

In [87]:
aa = calc_weight_delta(train, test)

In [92]:
aa.to_dataframe()

pm25   rh2   t2  u10  v10
gtime               ltime sta   vweight                              
2016-01-01 20:00:00 0     1008A 1_1_1_1_1  129.4  51.1  0.1 -1.1  0.6
                    1     1008A 1_1_1_1_1  146.6  29.3  4.8  1.7  0.1
                    2     1008A 1_1_1_1_1  153.4  22.1  5.6 -0.8 -0.5
                    3     1008A 1_1_1_1_1   87.3  11.2  6.5 -1.6  1.2
                    4     1008A 1_1_1_1_1   19.4   3.4  7.8 -2.0 -0.3
...                                          ...   ...  ...  ...  ...
2018-12-31 20:00:00 92    1008A 1_1_1_1_1  -16.2  14.0  0.2 -0.2 -2.5
                    93    1008A 1_1_1_1_1  -14.9   9.1  0.8  0.6 -1.5
                    94    1008A 1_1_1_1_1  -18.9   9.1  0.8  0.8 -1.6
                    95    1008A 1_1_1_1_1   -9.8  11.1  0.5  0.6 -1.7
                    96    1008A 1_1_1_1_1    0.3  12.9  0.3  0.4  0.1

[106021 rows x 5 columns]

In [27]:
windex=vweight.index

In [42]:
a,b = np.meshgrid(tindex.values, windex)

In [43]:
np.column_stack((a.ravel(), b.ravel())).shape

(212042, 2)

In [60]:
np.recarray(tindex.values, formats=['str', 'float64', 'int'])

ValueError: maximum supported dimension for an ndarray is 32, found 106021

In [61]:
vweight.to_xarray()

<xarray.Dataset>
Dimensions:  (index: 2)
Coordinates:
  * index    (index) object '1' '2'
Data variables:
    pm25     (index) float64 1.0 1.0
    rh2      (index) float64 1.0 1.0
    t2       (index) float64 1.0 1.0
    u10      (index) float64 1.0 1.0
    v10      (index) float64 1.0 1.0

In [64]:
end = train.to_xarray() * vweight.to_xarray()
end

<xarray.Dataset>
Dimensions:  (gtime: 1093, index: 2, ltime: 97, sta: 1)
Coordinates:
  * sta      (sta) object '1008A'
  * gtime    (gtime) datetime64[ns] 2016-01-01T20:00:00 ... 2018-12-31T20:00:00
  * ltime    (ltime) int64 0 1 2 3 4 5 6 7 8 9 ... 87 88 89 90 91 92 93 94 95 96
  * index    (index) object '1' '2'
Data variables:
    pm25     (sta, gtime, ltime, index) float64 172.9 172.9 183.4 ... 47.8 47.8
    rh2      (sta, gtime, ltime, index) float64 84.4 84.4 82.0 ... 38.7 38.7
    t2       (sta, gtime, ltime, index) float64 269.2 269.2 ... 267.5 267.5
    u10      (sta, gtime, ltime, index) float64 -0.3 -0.3 -0.2 ... 0.5 0.4 0.4
    v10      (sta, gtime, ltime, index) float64 -0.2 -0.2 -0.8 ... -1.0 0.0 0.0

In [72]:
(end.rolling(dim={'ltime':3}, min_periods =2) ** 2).mean()

TypeError: unsupported operand type(s) for ** or pow(): 'DatasetRolling' and 'int'

In [75]:
end ** 2

<xarray.Dataset>
Dimensions:  (gtime: 1093, index: 2, ltime: 97, sta: 1)
Coordinates:
  * index    (index) object '1' '2'
  * gtime    (gtime) datetime64[ns] 2016-01-01T20:00:00 ... 2018-12-31T20:00:00
  * sta      (sta) object '1008A'
  * ltime    (ltime) int64 0 1 2 3 4 5 6 7 8 9 ... 87 88 89 90 91 92 93 94 95 96
Data variables:
    pm25     (sta, gtime, ltime, index) float64 2.989e+04 ... 2.285e+03
    rh2      (sta, gtime, ltime, index) float64 7.123e+03 ... 1.498e+03
    t2       (sta, gtime, ltime, index) float64 7.247e+04 ... 7.156e+04
    u10      (sta, gtime, ltime, index) float64 0.09 0.09 0.04 ... 0.16 0.16
    v10      (sta, gtime, ltime, index) float64 0.04 0.04 0.64 ... 1.0 0.0 0.0

# 时间权重